In [1]:
import tensorflow as tf
from tensorflow.keras import layers 
from tensorflow.keras.regularizers import l2
from sklearn import datasets
import sklearn
import pandas as pd
import numpy as np
print(tf.__version__)

2.1.0


In [2]:
# import gzip
# import shutil
# # C:\Users\allro\JupyterNotebook\QTW\Data
# with gzip.open('C:\\Users\\allro\\JupyterNotebook\\QTW\\Data\\HIGGS.csv.gz', 'rb') as f_in:
#    with open('C:\\Users\\allro\\JupyterNotebook\\QTW\\Data\\HIGGS.csv', 'wb') as f_out:
#        shutil.copyfileobj(f_in, f_out)

In [5]:
#https://archive.ics.uci.edu/ml/datasets/HIGGS#
df = pd.read_csv("../../HIGGS.csv", header=None)

In [6]:
df.columns =['target', 'lepton_ph', 'lepton_eta', 'lepton_phi','missing_energy_magnitude','missing_energy_phi',
             'jet_1_pt','jet_1_eta','jet_1_phi','jet_1_btag','jet_2_pt','jet_2_eta','jet_2_phi','jet_2_btag',
             'jet_3_pt','jet_3_eta','jet_3_phi','jet_3_btag','jet_4_pt','jet_4_eta','jet_4_phi','jet_4_btag',
             'm_jj','m_jjj','m_lv','m_jlv','m_bb','m_wbb','m_wwbb'] 

In [7]:
# Print out the data types
df.info(verbose = True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11000000 entries, 0 to 10999999
Data columns (total 29 columns):
target                      float64
lepton_ph                   float64
lepton_eta                  float64
lepton_phi                  float64
missing_energy_magnitude    float64
missing_energy_phi          float64
jet_1_pt                    float64
jet_1_eta                   float64
jet_1_phi                   float64
jet_1_btag                  float64
jet_2_pt                    float64
jet_2_eta                   float64
jet_2_phi                   float64
jet_2_btag                  float64
jet_3_pt                    float64
jet_3_eta                   float64
jet_3_phi                   float64
jet_3_btag                  float64
jet_4_pt                    float64
jet_4_eta                   float64
jet_4_phi                   float64
jet_4_btag                  float64
m_jj                        float64
m_jjj                       float64
m_lv                 

In [6]:
df.head()

,target,lepton_ph,lepton_eta,lepton_phi,missing_energy_magnitude,missing_energy_phi,jet_1_pt,jet_1_eta,jet_1_phi,jet_1_btag,...,jet_4_eta,jet_4_phi,jet_4_btag,m_jj,m_jjj,m_lv,m_jlv,m_bb,m_wbb,m_wwbb
0,1.0,0.869293,-0.635082,0.225690,0.327470,-0.689993,0.754202,-0.248573,-1.092064,0.000000,...,-0.010455,-0.045767,3.101961,1.353760,0.979563,0.978076,0.920005,0.721657,0.988751,0.876678
1,1.0,0.907542,0.329147,0.359412,1.497970,-0.313010,1.095531,-0.557525,-1.588230,2.173076,...,-1.138930,-0.000819,0.000000,0.302220,0.833048,0.985700,0.978098,0.779732,0.992356,0.798343
2,1.0,0.798835,1.470639,-1.635975,0.453773,0.425629,1.104875,1.282322,1.381664,0.000000,...,1.128848,0.900461,0.000000,0.909753,1.108330,0.985692,0.951331,0.803252,0.865924,0.780118
3,0.0,1.344385,-0.876626,0.935913,1.992050,0.882454,1.786066,-1.646778,-0.942383,0.000000,...,-0.678379,-1.360356,0.000000,0.946652,1.028704,0.998656,0.728281,0.869200,1.026736,0.957904
4,1.0,1.105009,0.321356,1.522401,0.882808,-1.205349,0.681466,-1.070464,-0.921871,0.000000,...,-0.373566,0.113041,0.000000,0.755856,1.361057,0.986610,0.838085,1.133295,0.872245,0.808487


In [8]:
#subsample the data
train = df.sample(n=2600000, random_state=123)

In [9]:
#figure out which columns have values strictly greater than 0, for scaler purposes
pd.set_option("display.max_rows", 500, "display.max_columns", None)
df.describe()

,target,lepton_ph,lepton_eta,lepton_phi,missing_energy_magnitude,missing_energy_phi,jet_1_pt,jet_1_eta,jet_1_phi,jet_1_btag,jet_2_pt,jet_2_eta,jet_2_phi,jet_2_btag,jet_3_pt,jet_3_eta,jet_3_phi,jet_3_btag,jet_4_pt,jet_4_eta,jet_4_phi,jet_4_btag,m_jj,m_jjj,m_lv,m_jlv,m_bb,m_wbb,m_wwbb
count,1.100000e+07,1.100000e+07,1.100000e+07,1.100000e+07,1.100000e+07,1.100000e+07,1.100000e+07,1.100000e+07,1.100000e+07,1.100000e+07,1.100000e+07,1.100000e+07,1.100000e+07,1.100000e+07,1.100000e+07,1.100000e+07,1.100000e+07,1.100000e+07,1.100000e+07,1.100000e+07,1.100000e+07,1.100000e+07,1.100000e+07,1.100000e+07,1.100000e+07,1.100000e+07,1.100000e+07,1.100000e+07,1.100000e+07
mean,5.299203e-01,9.914658e-01,-8.297618e-06,-1.327225e-05,9.985364e-01,2.613459e-05,9.909152e-01,-2.027520e-05,7.716199e-06,9.999687e-01,9.927294e-01,-1.026444e-05,-2.076887e-05,1.000008e+00,9.922591e-01,1.459561e-05,3.678632e-06,1.000011e+00,9.861087e-01,-5.756954e-06,1.744903e-05,1.000000e+00,1.034290e+00,1.024805e+00,1.050554e+00,1.009742e+00,9.729596e-01,1.033036e+00,9.598120e-01
std,4.991040e-01,5.653777e-01,1.008827e+00,1.006346e+00,6.000185e-01,1.006326e+00,4.749747e-01,1.009303e+00,1.005901e+00,1.027808e+00,4.999939e-01,1.009331e+00,1.006154e+00,1.049398e+00,4.876623e-01,1.008747e+00,1.006305e+00,1.193676e+00,5.057777e-01,1.007694e+00,1.006366e+00,1.400209e+00,6.746354e-01,3.808074e-01,1.645763e-01,3.974453e-01,5.254063e-01,3.652556e-01,3.133378e-01
min,0.000000e+00,2.746966e-01,-2.434976e+00,-1.742508e+00,2.370088e-04,-1.743944e+00,1.375024e-01,-2.969725e+00,-1.741237e+00,0.000000e+00,1.889811e-01,-2.913090e+00,-1.742372e+00,0.000000e+00,2.636076e-01,-2.729663e+00,-1.742069e+00,0.000000e+00,3.653542e-01,-2.497265e+00,-1.742691e+00,0.000000e+00,7.507046e-02,1.986757e-01,8.304866e-02,1.320062e-01,4.786215e-02,2.951122e-01,3.307214e-01
25%,0.000000e+00,5.907533e-01,-7.383225e-01,-8.719308e-01,5.768156e-01,-8.712081e-01,6.789927e-01,-6.872450e-01,-8.680962e-01,0.000000e+00,6.564608e-01,-6.944718e-01,-8.701791e-01,0.000000e+00,6.508527e-01,-6.998083e-01,-8.711343e-01,0.000000e+00,6.177673e-01,-7.141902e-01,-8.714789e-01,0.000000e+00,7.906095e-01,8.462266e-01,9.857525e-01,7.675732e-01,6.738168e-01,8.193964e-01,7.703901e-01
50%,1.000000e+00,8.533714e-01,-5.415563e-05,-2.410638e-04,8.916277e-01,2.125454e-04,8.948193e-01,-2.543566e-05,5.813991e-05,1.086538e+00,8.901377e-01,6.027267e-05,3.514990e-04,0.000000e+00,8.972494e-01,1.728937e-04,-7.519117e-04,0.000000e+00,8.682333e-01,3.721330e-04,-2.642369e-04,0.000000e+00,8.949304e-01,9.506853e-01,9.897798e-01,9.165110e-01,8.733798e-01,9.473447e-01,8.719701e-01
75%,1.000000e+00,1.236226e+00,7.382142e-01,8.709940e-01,1.293056e+00,8.714708e-01,1.170740e+00,6.871941e-01,8.683126e-01,2.173076e+00,1.201875e+00,6.945924e-01,8.698727e-01,2.214872e+00,1.221798e+00,7.001541e-01,8.713947e-01,2.548224e+00,1.220930e+00,7.141017e-01,8.716055e-01,3.101961e+00,1.024730e+00,1.083493e+00,1.020528e+00,1.142226e+00,1.138439e+00,1.140458e+00,1.059248e+00
max,1.000000e+00,1.209891e+01,2.434868e+00,1.743236e+00,1.539682e+01,1.743257e+00,9.940391e+00,2.969674e+00,1.741454e+00,2.173076e+00,1.164708e+01,2.913210e+00,1.743175e+00,2.214872e+00,1.470899e+01,2.730009e+00,1.742884e+00,2.548224e+00,1.288257e+01,2.498009e+00,1.743372e+00,3.101961e+00,4.019237e+01,2.037278e+01,7.992739e+00,1.426244e+01,1.776285e+01,1.149652e+01,8.374498e+00


In [10]:
train.shape

(2600000, 29)

In [17]:
y = train['target']
pre_X = train.loc[:, df.columns != 'target']

from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
scaled_train = scaler.fit_transform(pre_X)
X = pd.DataFrame(data=scaled_train, columns=pre_X.columns)

In [13]:
#not complete, need columns where mean=1 and stdev=1
from sklearn.preprocessing import StandardScaler

#these columns scale where mean=0 and stdev=1
to_scale1 = pre_X[['lepton_eta','lepton_phi','missing_energy_phi','jet_1_eta','jet_1_phi','jet_2_eta','jet_2_phi','jet_3_eta','jet_3_phi','jet_4_eta','jet_4_phi']]
scaler = StandardScaler()
scaled_train = scaler.fit_transform(to_scale1)


In [16]:
scaled_train_df = pd.DataFrame(scaled_train, columns=['lepton_eta','lepton_phi','missing_energy_phi','jet_1_eta','jet_1_phi','jet_2_eta','jet_2_phi','jet_3_eta','jet_3_phi','jet_4_eta','jet_4_phi'])
scaled_train_df.describe()

,lepton_eta,lepton_phi,missing_energy_phi,jet_1_eta,jet_1_phi,jet_2_eta,jet_2_phi,jet_3_eta,jet_3_phi,jet_4_eta,jet_4_phi
count,2.600000e+06,2.600000e+06,2.600000e+06,2.600000e+06,2.600000e+06,2.600000e+06,2.600000e+06,2.600000e+06,2.600000e+06,2.600000e+06,2.600000e+06
mean,4.131178e-17,8.924613e-17,-3.083901e-17,4.807138e-18,-4.718029e-17,-8.113305e-17,-2.608358e-17,2.121924e-16,-2.010950e-16,-8.899181e-17,2.780400e-17
std,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00
min,-2.414452e+00,-1.731104e+00,-1.731925e+00,-2.942199e+00,-1.730027e+00,-2.885415e+00,-1.732408e+00,-2.705312e+00,-1.730599e+00,-2.479189e+00,-1.731107e+00
25%,-7.321584e-01,-8.659604e-01,-8.658190e-01,-6.819477e-01,-8.632094e-01,-6.889230e-01,-8.647062e-01,-6.935506e-01,-8.659706e-01,-7.085818e-01,-8.667571e-01
50%,-1.386904e-04,-1.365973e-03,2.090639e-04,-1.125500e-03,-7.977719e-04,9.082319e-04,8.939700e-04,1.093501e-03,3.138265e-04,6.528141e-04,-1.998949e-04
75%,7.318811e-01,8.655338e-01,8.662048e-01,6.806777e-01,8.628153e-01,6.888153e-01,8.641829e-01,6.939334e-01,8.661466e-01,7.082342e-01,8.659056e-01
max,2.414175e+00,1.732883e+00,1.732090e+00,2.941910e+00,1.731837e+00,2.885307e+00,1.732988e+00,2.705695e+00,1.731326e+00,2.478841e+00,1.731909e+00


In [18]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=500000, random_state=1776)

In [19]:
# y = y_train.to_numpy()
# type(np.unique(y)[0])

y_train = y_train.astype(np.int64)
y_test = y_test.astype(np.int64)

In [20]:
x_train.shape

(2100000, 28)

In [21]:
X.describe()

,lepton_ph,lepton_eta,lepton_phi,missing_energy_magnitude,missing_energy_phi,jet_1_pt,jet_1_eta,jet_1_phi,jet_1_btag,jet_2_pt,jet_2_eta,jet_2_phi,jet_2_btag,jet_3_pt,jet_3_eta,jet_3_phi,jet_3_btag,jet_4_pt,jet_4_eta,jet_4_phi,jet_4_btag,m_jj,m_jjj,m_lv,m_jlv,m_bb,m_wbb,m_wwbb
count,2.600000e+06,2.600000e+06,2.600000e+06,2.600000e+06,2.600000e+06,2.600000e+06,2.600000e+06,2.600000e+06,2.600000e+06,2.600000e+06,2.600000e+06,2.600000e+06,2.600000e+06,2.600000e+06,2.600000e+06,2.600000e+06,2.600000e+06,2.600000e+06,2.600000e+06,2.600000e+06,2.600000e+06,2.600000e+06,2.600000e+06,2.600000e+06,2.600000e+06,2.600000e+06,2.600000e+06,2.600000e+06
mean,6.290482e-02,5.000287e-01,4.997432e-01,7.768960e-02,4.999762e-01,1.011576e-01,5.000246e-01,4.997386e-01,4.598750e-01,8.221750e-02,5.000093e-01,4.999163e-01,4.513519e-01,5.045377e-02,4.999646e-01,4.998950e-01,3.924435e-01,4.962456e-02,5.000351e-01,4.998842e-01,3.226283e-01,3.259405e-02,5.394110e-02,1.390821e-01,7.079912e-02,6.737304e-02,8.381950e-02,9.688168e-02
std,4.955006e-02,2.070983e-01,2.886848e-01,4.672872e-02,2.886824e-01,5.629750e-02,1.699493e-01,2.888618e-01,4.729015e-01,5.114958e-02,1.732886e-01,2.885673e-01,4.737810e-01,3.374349e-02,1.848085e-01,2.888566e-01,4.684384e-01,4.046157e-02,2.016931e-01,2.887657e-01,4.514692e-01,2.291845e-02,2.587445e-02,2.452874e-02,3.545985e-02,3.837583e-02,4.317710e-02,5.010340e-02
min,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
25%,2.775815e-02,3.484000e-01,2.497537e-01,4.483757e-02,2.500295e-01,6.416143e-02,3.841281e-01,2.503904e-01,0.000000e+00,4.780415e-02,3.806269e-01,2.503904e-01,0.000000e+00,2.680753e-02,3.717906e-01,2.497537e-01,0.000000e+00,2.016529e-02,3.571191e-01,2.495945e-01,0.000000e+00,2.429879e-02,4.180044e-02,1.294109e-01,4.918208e-02,4.549617e-02,5.856219e-02,6.658847e-02
50%,5.079630e-02,5.000000e-01,4.993489e-01,6.937425e-02,5.000365e-01,8.977824e-02,4.998333e-01,4.995081e-01,5.000000e-01,7.171910e-02,5.001667e-01,5.001743e-01,0.000000e+00,4.387682e-02,5.001667e-01,4.999856e-01,0.000000e+00,4.017501e-02,5.001667e-01,4.998265e-01,0.000000e+00,2.784232e-02,4.888833e-02,1.300110e-01,6.249474e-02,6.011345e-02,7.367504e-02,8.282983e-02
75%,8.435011e-02,6.516000e-01,7.496096e-01,1.005773e-01,7.500342e-01,1.224505e-01,6.157052e-01,7.489729e-01,1.000000e+00,1.036014e-01,6.193731e-01,7.492913e-01,1.000000e+00,6.638056e-02,6.282094e-01,7.500871e-01,1.000000e+00,6.841030e-02,6.428810e-01,7.499280e-01,1.000000e+00,3.226186e-02,5.793053e-02,1.346274e-01,8.262032e-02,7.949804e-02,9.648749e-02,1.127462e-01
max,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00


In [39]:
from tensorflow.keras import initializers

model = tf.keras.Sequential()
# input
model.add(tf.keras.Input(shape=(28,)))
# hidden
model.add(layers.Dense(300,
                       activation='tanh', 
                       kernel_initializer=initializers.RandomNormal(stddev=0.1)))  
model.add(layers.Dense(300,
                       activation='tanh', 
                       kernel_initializer=initializers.RandomNormal(stddev=0.05)))
model.add(layers.Dense(300,
                       activation='tanh', 
                       kernel_initializer=initializers.RandomNormal(stddev=0.05)))
model.add(layers.Dense(300,
                       activation='tanh', 
                       kernel_initializer=initializers.RandomNormal(stddev=0.05)))
model.add(layers.Dense(1,
                       activation='sigmoid',
                       kernel_initializer=initializers.RandomNormal(stddev=0.001)))  


In [40]:
from tensorflow.keras import optimizers

opt = tf.keras.optimizers.SGD(learning_rate=0.05, momentum=1e-5)

model.compile(optimizer=opt,
              loss='binary_crossentropy',
              metrics=['AUC'])

In [41]:
%%time

model.fit(x_train, y_train, epochs=250, validation_data=(x_test,y_test), batch_size=100)

Train on 2100000 samples, validate on 500000 samples
Epoch 1/250
2100000/2100000 [==============================] - 183s 87us/sample - loss: 0.6558 - AUC: 0.6438 - val_loss: 0.6422 - val_AUC: 0.6778
Epoch 2/250
2100000/2100000 [==============================] - 180s 86us/sample - loss: 0.6426 - AUC: 0.6699 - val_loss: 0.6362 - val_AUC: 0.6837
Epoch 3/250
2100000/2100000 [==============================] - 178s 85us/sample - loss: 0.6298 - AUC: 0.6933 - val_loss: 0.6182 - val_AUC: 0.7212
Epoch 4/250
2100000/2100000 [==============================] - 180s 86us/sample - loss: 0.6140 - AUC: 0.7177 - val_loss: 0.6130 - val_AUC: 0.7290
Epoch 5/250
2100000/2100000 [==============================] - 179s 85us/sample - loss: 0.6007 - AUC: 0.7359 - val_loss: 0.5878 - val_AUC: 0.7523
Epoch 6/250
2100000/2100000 [==============================] - 182s 87us/sample - loss: 0.5911 - AUC: 0.7474 - val_loss: 0.5875 - val_AUC: 0.7529
Epoch 7/250
2100000/2100000 [==============================] - 175s 83u

2100000/2100000 [==============================] - 146s 69us/sample - loss: 0.4960 - AUC: 0.8359 - val_loss: 0.5056 - val_AUC: 0.8286
Epoch 57/250
2100000/2100000 [==============================] - 143s 68us/sample - loss: 0.4953 - AUC: 0.8364 - val_loss: 0.5026 - val_AUC: 0.8310
Epoch 58/250
2100000/2100000 [==============================] - 140s 67us/sample - loss: 0.4947 - AUC: 0.8368 - val_loss: 0.4950 - val_AUC: 0.8372
Epoch 59/250
2100000/2100000 [==============================] - 152s 72us/sample - loss: 0.4944 - AUC: 0.8371 - val_loss: 0.5005 - val_AUC: 0.8329
Epoch 60/250
2100000/2100000 [==============================] - 145s 69us/sample - loss: 0.4938 - AUC: 0.8375 - val_loss: 0.4942 - val_AUC: 0.8375
Epoch 61/250
2100000/2100000 [==============================] - 138s 66us/sample - loss: 0.4931 - AUC: 0.8380 - val_loss: 0.4953 - val_AUC: 0.8365
Epoch 62/250
2100000/2100000 [==============================] - 141s 67us/sample - loss: 0.4926 - AUC: 0.8384 - val_loss: 0.4960 - 

Epoch 112/250
2100000/2100000 [==============================] - 145s 69us/sample - loss: 0.4729 - AUC: 0.8527 - val_loss: 0.4882 - val_AUC: 0.8427
Epoch 113/250
2100000/2100000 [==============================] - 142s 68us/sample - loss: 0.4727 - AUC: 0.8529 - val_loss: 0.4894 - val_AUC: 0.8413
Epoch 114/250
2100000/2100000 [==============================] - 144s 68us/sample - loss: 0.4722 - AUC: 0.8532 - val_loss: 0.4926 - val_AUC: 0.8388
Epoch 115/250
2100000/2100000 [==============================] - 140s 67us/sample - loss: 0.4719 - AUC: 0.8534 - val_loss: 0.4869 - val_AUC: 0.8432
Epoch 116/250
2100000/2100000 [==============================] - 140s 67us/sample - loss: 0.4716 - AUC: 0.8536 - val_loss: 0.4899 - val_AUC: 0.8413
Epoch 117/250
2100000/2100000 [==============================] - 140s 67us/sample - loss: 0.4713 - AUC: 0.8538 - val_loss: 0.4913 - val_AUC: 0.8402
Epoch 118/250
2100000/2100000 [==============================] - 141s 67us/sample - loss: 0.4710 - AUC: 0.8541 -

2100000/2100000 [==============================] - 141s 67us/sample - loss: 0.4557 - AUC: 0.8645 - val_loss: 0.4954 - val_AUC: 0.8390
Epoch 168/250
2100000/2100000 [==============================] - 142s 68us/sample - loss: 0.4555 - AUC: 0.8646 - val_loss: 0.4953 - val_AUC: 0.8401
Epoch 169/250
2100000/2100000 [==============================] - 140s 67us/sample - loss: 0.4553 - AUC: 0.8647 - val_loss: 0.4960 - val_AUC: 0.8383
Epoch 170/250
2100000/2100000 [==============================] - 142s 67us/sample - loss: 0.4549 - AUC: 0.8650 - val_loss: 0.4989 - val_AUC: 0.8384
Epoch 171/250
2100000/2100000 [==============================] - 141s 67us/sample - loss: 0.4546 - AUC: 0.8652 - val_loss: 0.4954 - val_AUC: 0.8386
Epoch 172/250
2100000/2100000 [==============================] - 141s 67us/sample - loss: 0.4542 - AUC: 0.8654 - val_loss: 0.4954 - val_AUC: 0.8389
Epoch 173/250
2100000/2100000 [==============================] - 141s 67us/sample - loss: 0.4539 - AUC: 0.8656 - val_loss: 0.4

2100000/2100000 [==============================] - 140s 67us/sample - loss: 0.4411 - AUC: 0.8739 - val_loss: 0.5084 - val_AUC: 0.8326
Epoch 223/250
2100000/2100000 [==============================] - 138s 66us/sample - loss: 0.4407 - AUC: 0.8741 - val_loss: 0.5110 - val_AUC: 0.8326
Epoch 224/250
2100000/2100000 [==============================] - 142s 68us/sample - loss: 0.4405 - AUC: 0.8743 - val_loss: 0.5100 - val_AUC: 0.8308
Epoch 225/250
2100000/2100000 [==============================] - 140s 66us/sample - loss: 0.4402 - AUC: 0.8745 - val_loss: 0.5164 - val_AUC: 0.8265
Epoch 226/250
2100000/2100000 [==============================] - 140s 67us/sample - loss: 0.4402 - AUC: 0.8745 - val_loss: 0.5065 - val_AUC: 0.8336
Epoch 227/250
2100000/2100000 [==============================] - 143s 68us/sample - loss: 0.4400 - AUC: 0.8746 - val_loss: 0.5111 - val_AUC: 0.8319
Epoch 228/250
2100000/2100000 [==============================] - 143s 68us/sample - loss: 0.4397 - AUC: 0.8747 - val_loss: 0.5